In [30]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import pandas as pd
from sklearn.ensemble import IsolationForest

In [31]:
#Read the data
final_data_short = pd.read_csv('final_data_short_sample.csv')
final_data_short.fillna(0, inplace=True) 
data_labels = pd.read_csv('labelled_data.csv')

# Assuming the first column in final_data_short is consumer_number
final_data_short.set_index('consumer_number', inplace=True)
data_labels.set_index('consumer_number', inplace=True)

# Extract the features for the labeled data
labeled_features = final_data_short.loc[data_labels.index]
y_labeled = data_labels['fraud_status']
print(labeled_features)

                 01/01/22 0:00  01/01/22 1:00  01/01/22 2:00  01/01/22 3:00  \
consumer_number                                                               
1                     1.375976       1.019364       0.887116       0.833463   
2                     1.379896       1.059830       0.933796       0.883621   
3                     1.398051       0.971885       0.799060       0.741189   
4                     1.415157       0.974334       0.796582       0.735028   
5                     1.409090       0.958425       0.772773       0.715845   
...                        ...            ...            ...            ...   
296                   1.395289       1.062266       0.932044       0.881525   
297                   1.417902       1.094939       0.975381       0.935705   
298                   1.241831       0.930760       0.805257       0.756684   
299                   1.240265       0.911813       0.783575       0.734499   
300                   1.143511       0.868656       

In [48]:
# Hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100],
    'max_samples': ['auto', 0.5, 0.75],
    'contamination': np.arange(0.0001, 0.5, 0.000001),
    'random_state': [23]
}

# Initialize GridSearchCV with Isolation Forest
grid_search = GridSearchCV(IsolationForest(), param_grid, scoring=make_scorer(recall_score), cv=3, n_jobs=-1)
grid_search.fit(labeled_features, y_labeled)

# Retrieve the best hyperparameters
best_params = grid_search.best_params_
best_params

/Users/simar/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/simar/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/simar/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/simar/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/simar/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/Users/simar/opt/anaconda3/lib/pyth

{'contamination': 0.05,
 'max_samples': 'auto',
 'n_estimators': 50,
 'random_state': 23}

In [43]:
# Load the datasets
final_data_short = pd.read_csv('final_data_short.csv')

# Preprocess the full dataset
final_data_short.fillna(0, inplace=True)  # Fill missing values

# Assuming the first column is 'consumer_number', and we don't use it for anomaly detection
final_data_short.drop('consumer_number', axis=1, inplace=True)

# Using the provided best parameters to create and train the model
best_params = {
    'contamination': 0.05,
    'max_samples': 'auto',
    'n_estimators': 50,
    'random_state': 23
}

# Create and fit the Isolation Forest model
model = IsolationForest(**best_params)
model.fit(final_data_short)

# Predict anomalies on the full dataset
predictions = model.predict(final_data_short)
predictions = np.where(predictions == -1, 1, 0)  # Convert to binary labels (1 for anomaly/fraud)

# Adding predictions to the dataset
final_data_short['anomaly_prediction'] = predictions
print(final_data_short['anomaly_prediction'])

/Users/simar/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


0       0
1       0
2       0
3       0
4       0
       ..
2800    0
2801    0
2802    0
2803    0
2804    0
Name: anomaly_prediction, Length: 2805, dtype: int64


In [44]:
data_labels = pd.read_csv('labelled_data.csv')
accuracy = accuracy_score(data_labels['fraud_status'], predictions[0:300])


In [45]:
print(accuracy)

0.9


In [19]:
recall=recall_score(data_labels['fraud_status'], predictions[0:300], average='macro')

In [20]:
print(recall)

0.8013936720142601
